In [1]:
import os
import cv2
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.metrics import f1_score

In [3]:
class Rectangle__:
    def __init__(self, xmin, ymin, width, height):
        self.xmin = xmin
        self.ymin = ymin
        self.width = width
        self.height = height
        
    def describe(self):
        return "xmin : {}, ymin : {}, width : {}, height : {}".format(self.xmin, self.ymin, self.width, self.height)
    # ########## intersects(self, other)
    # Checking overlapped rects
    # Reference : https://stackoverflow.com/questions/40795709/checking-whether-two-rectangles-overlap-in-python-using-two-bottom-left-corners
    def intersects(self, other):
        if (self.xmin > other.xmin + other.width or self.xmin + self.width < other.xmin or
            self.ymin > other.ymin + other.height or self.ymin + self.height < other.ymin):
            return False
        return True
    
    def sumOfIntersect(self, other):
        if self.intersects(other):
            print("Two rects are intersecting!")
            left_up_x = max(self.xmin, other.xmin)
            left_up_y = max(self.ymin, other.ymin)
            right_down_x = min(self.xmin + self.width, other.xmin + other.width)
            right_down_y = min(self.ymin + self.height, other.ymin + other.height)
            width = right_down_x - left_up_x
            height = right_down_y - left_up_y
            return width * height
        return

        
    # ########## def isEmptyRect():
    # Checks if a rect is empty
    def isEmpty():
        return not (self.width > 0 or self.height > 0)
    # ##########

    # ########## def sum(self):
    # Calculats a sum of a rect
    # ##########
    def sum(self):
        return self.width * self.height

def isRectOkay(xmin, ymin, w_img, h_img, rect_w, rect_h):
    return not xmin < 0 or ymin < 0 or xmin >= w_img or ymin >= h_img or rect_w == 0 or rect_h == 0

def determine(IoU):
    if IoU >= 0.5 : 
        return "TP"
    if IoU < 0.5 : 
        return "FP"

path_dir = './Roboflow_test/'
# path_dir = './Roboflow_split/'
# path_dir = './42/'
# path_dir = './Roboflow_TEST_DATA/'
txt_list = []
jpg_list = []
for file in os.listdir(path_dir):
    if "txt" in file:
        if "converted" not in file:
            txt_list.append(file)
    if "jpg" in file:
        jpg_list.append(file)

# print("txt_list :", txt_list)
# print("jpg_list :", jpg_list)

In [4]:
tree_cascade=cv2.CascadeClassifier("./XML/cascade_final.xml") ##path of cascade file
# tree_cascade=cv2.CascadeClassifier("./XML/1213redceder_roboflow_stage20.xml") ##path of cascade file

In [5]:
import csv
filename = 'caluateIoU.csv'
f = open(filename,'w', newline='')
wr = csv.writer(f)

In [6]:
numTP = 0
numFP = 0
numFN = 0
numDetect = 0
numAns = 0
i=0
for txtFile in txt_list:
    # ########## [+] Fetching Answer Data
    txtPath = path_dir + txtFile

    # Opening Txt file
    file = open(txtPath, 'r')
    Lines = file.readlines()
    if not Lines: # If the list is empty, continue
        continue
    
    print("\n[+] {} Start!".format(txtFile))
    img = cv2.imread(txtPath[:-4] + ".jpg", cv2.IMREAD_COLOR)
    
    count = 0
    haar_coord = ""
    ans_rects = []
    for line in Lines: # Strips the newline character
        count += 1
        words = line.split()
        (classes, xcenter, ycenter, w, h) = list(map(float, words))
        (h_img, w_img, _) = img.shape
        (rect_h, rect_w) = (int(h * h_img), int(w * w_img))
        (xmin, ymin) = (int(xcenter * w_img - (rect_w / 2)), int(ycenter * h_img - (rect_h / 2)))
        if not isRectOkay(xmin, ymin, w_img, h_img, rect_w, rect_h):
            print("Problem!!! [xmin : {}, ymin : {}, rect_w : {}, rect_h : {}]".format(xmin, ymin, rect_w, rect_h))
            continue
        else:
            rect = Rectangle__(xmin, ymin, rect_w, rect_h)
            ans_rects.append(rect)
        
    gatheredLine = "{} {}".format(txtPath[:-4] + ".jpg", count) + haar_coord + '\n'

    print("gatheredLine : ", gatheredLine)
    # ########## [+] Fetching Answer Data
    
    # ########## Detecting the Red Cedar ##########
    detected = []
    
    # convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (25, 52, 72), (102, 255, 255))
#     mask = cv2.inRange(hsv, (25, 15, 100), (80, 255, 255))
    img_result = cv2.bitwise_and(img, img, mask=mask)
    # 
    gray = cv2.cvtColor(img_result, cv2.COLOR_BGR2GRAY)
    trees = tree_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3)#, minSize=(20,20))
    for (x,y,w,h) in trees:
        detected.append(Rectangle__(x, y, w, h))
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),1)
    
    cv2.imwrite("C:/test_result/result_%d.jpg" % i, img)
    i+=1
    
    # ########## detected : List of Rects ##########
    detected_result = []
    numAns += len(ans_rects)
    numDetect += len(detected)
    print("num of ans_rects : ", len(ans_rects))
    print("num of detected : ", len(detected))
    for candidate in detected:
        print("candidate :", candidate.describe())
        flag = 0
        for ans in ans_rects:
            print("\tans :", ans.describe())
            sumOfArea = ans.sumOfIntersect(candidate)
            if sumOfArea:
                flag = 1
                totalArea = ans.sum() + candidate.sum() - sumOfArea
                IoU = sumOfArea / totalArea
                detected_result.append(IoU)
                if(determine(IoU)=="TP"): 
                    numTP += 1
                else:
                    numFP += 1
                break
#                 wr.writerow([IoU, determine(IoU)])
#                 if IoU == 0 : numFN += 1
#                 print("IoU : {}".format(IoU))
#                 image = cv2.rectangle(img, (ans.xmin, ans.ymin), (ans.xmin + ans.rect_w, ans.ymin + ans.rect_h), (255, 255, 255), 1)
#                 image2 = cv2.rectangle(image, (detected.xmin, detected.ymin), (detected.xmin + detected.rect_w, detected.ymin + detected.rect_h), (255, 255, 255), 1)
#                 plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGBA)) # Because OpenCV stores images in BGR order instead of RGB.
#                 plt.show()
        if flag == 0 :
            detected_result.append(0.0)
            numFP += 1
#             wr.writerow([0.0, "FP"])
        flag = 0
    for ans in ans_rects:
        flag = 0
        for candidate in detected:
            sumOfArea = ans.sumOfIntersect(candidate)
            if sumOfArea:
                flag = 1
        if flag == 0 :
            numFN += 1
        flag = 0
        
    print("detected_result : {}".format(detected_result))
    
    print("[-] {} DONE!".format(txtFile))
    
# with open(path_dir + "converted.txt", 'w') as f:
#     for line in result:
#         f.write(line)

print("Detected : ", numDetect)
print("Answers : ", numAns)
# numTP = numTP * 2
# numFP = int(numFP/2)
# numFN = int(numFN/2)
print("TP ", numTP)
print("FP ", numFP)
print("FN ", numFN)
precision = numTP/(numTP+numFP)
recall = numTP/(numTP+numFN)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 Score = ", 2*precision*recall/(precision+recall))
print("[ !!!END!!! ]")


[+] MAX_0042__MP4-0_jpg.rf.477ade0136fd03ba223e6bd675a0aece.txt Start!
gatheredLine :  ./42/MAX_0042__MP4-0_jpg.rf.477ade0136fd03ba223e6bd675a0aece.jpg 1

num of ans_rects :  1
num of detected :  6
candidate : xmin : 116, ymin : 62, width : 45, height : 45
	ans : xmin : 349, ymin : 234, width : 9, height : 17
candidate : xmin : 137, ymin : 22, width : 27, height : 27
	ans : xmin : 349, ymin : 234, width : 9, height : 17
candidate : xmin : 132, ymin : 193, width : 26, height : 26
	ans : xmin : 349, ymin : 234, width : 9, height : 17
candidate : xmin : 104, ymin : 158, width : 29, height : 29
	ans : xmin : 349, ymin : 234, width : 9, height : 17
candidate : xmin : 134, ymin : 36, width : 25, height : 25
	ans : xmin : 349, ymin : 234, width : 9, height : 17
candidate : xmin : 97, ymin : 168, width : 31, height : 31
	ans : xmin : 349, ymin : 234, width : 9, height : 17
detected_result : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-] MAX_0042__MP4-0_jpg.rf.477ade0136fd03ba223e6bd675a0aece.txt DONE!

[

num of ans_rects :  3
num of detected :  24
candidate : xmin : 276, ymin : 170, width : 30, height : 30
	ans : xmin : 245, ymin : 213, width : 87, height : 202
	ans : xmin : 38, ymin : 291, width : 73, height : 105
	ans : xmin : 165, ymin : 365, width : 72, height : 50
candidate : xmin : 237, ymin : 242, width : 67, height : 67
	ans : xmin : 245, ymin : 213, width : 87, height : 202
Two rects are intersecting!
candidate : xmin : 7, ymin : 378, width : 28, height : 28
	ans : xmin : 245, ymin : 213, width : 87, height : 202
	ans : xmin : 38, ymin : 291, width : 73, height : 105
	ans : xmin : 165, ymin : 365, width : 72, height : 50
candidate : xmin : 339, ymin : 22, width : 26, height : 26
	ans : xmin : 245, ymin : 213, width : 87, height : 202
	ans : xmin : 38, ymin : 291, width : 73, height : 105
	ans : xmin : 165, ymin : 365, width : 72, height : 50
candidate : xmin : 154, ymin : 377, width : 33, height : 33
	ans : xmin : 245, ymin : 213, width : 87, height : 202
	ans : xmin : 38, ymi

num of ans_rects :  1
num of detected :  11
candidate : xmin : 253, ymin : 267, width : 27, height : 27
	ans : xmin : 232, ymin : 362, width : 116, height : 53
candidate : xmin : 253, ymin : 320, width : 46, height : 46
	ans : xmin : 232, ymin : 362, width : 116, height : 53
Two rects are intersecting!
candidate : xmin : 322, ymin : 217, width : 27, height : 27
	ans : xmin : 232, ymin : 362, width : 116, height : 53
candidate : xmin : 245, ymin : 380, width : 31, height : 31
	ans : xmin : 232, ymin : 362, width : 116, height : 53
Two rects are intersecting!
candidate : xmin : 15, ymin : 375, width : 29, height : 29
	ans : xmin : 232, ymin : 362, width : 116, height : 53
candidate : xmin : 22, ymin : 367, width : 26, height : 26
	ans : xmin : 232, ymin : 362, width : 116, height : 53
candidate : xmin : 283, ymin : 348, width : 49, height : 49
	ans : xmin : 232, ymin : 362, width : 116, height : 53
Two rects are intersecting!
candidate : xmin : 181, ymin : 135, width : 30, height : 30
	a

In [7]:
# cv2.inRange(hsv, (25, 52, 72), (102, 255, 255))
# color = [102, 255, 255] # 파란색
# print("hsv: ", hsv) # +_ 10

In [8]:
precision = 
recall = 0.3902439024390244

SyntaxError: invalid syntax (<ipython-input-8-92640e7bbf0f>, line 1)